<a href="https://colab.research.google.com/github/XiaoyangCC0022/ass2_biometric/blob/main/assignment_2_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![KU Leuven](./img/logo.png)
# Biometrics System Concepts
## Assignment 2: Fingerprint and Iris based Identification

**Name:** Joe Pesci |
**Student Nr:** KD6-3.7 |
**Date:** February 10, 1942
---

Implement and test a keypoint-based fingerprint and iris recognition system and fuse the two systems together.

A high-level description is provided with links to or hints of code snippets and libraries that you can reuse/adapt at your will (with proper referencing!).

This document is structured as below:  
* [I. Setting](#I.-Setting)
* [II. Fingerprint Recognition](#II.-Fingerprint-Recognition)
  1. Reading data
  2. Baseline model
    1. Choosing a similarity metric
    2. Constructing the similarity table
  3. Minutiae-Based Matching¶
    1. Image Enhancement
    2. Minutiae Detection
    3. Global Matching and Image Alignment
    4. Global Similarity Metric
    5. Validation
* [III. Iris Recognition](#III.-Iris-recognition)
  1. Reading data
  2. Biometric Iris Recogntion System
    1. Image enhancement
    2. Triplet Loss Encoder
    3. Validation
* [IV. Multimodal System](#IV-Multimodal-System)
  1. Score fusion
  2. Solve the murder case  
* [V. Assignment Instructions](#V.-Assignment-Instructions)


Code examples will be provided below. You can and are invited to adapt these at your will (different parameter settings, different choices of alogorithmic components, add external python files, ...). Try to keep things structured!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/BioM2')

In [ ]:
#to plot figures inline
%matplotlib inline

# OpenCV package
import cv2
# Standard array processing package
import numpy as np
# Plotting library
from matplotlib import pyplot as plt
# Setting the default colormap for pyplot
import matplotlib as mpl
mpl.rc('image', cmap='gray')
# File path processing package
from pathlib import Path

# Package for some simple biometric metrics.
# Of course you can use the code you have developed in the previous assignment
from sklearn.metrics import roc_curve
# Pickle allows to save and read intermediate results (similar to save and load in Matlab)
import pickle
# A visual progress bar library
from tqdm.notebook import tqdm as tqdm_notebook
# Allows us to generate markdown using python code
from IPython.display import Markdown
# Data analysis and manipulation tool
import pandas as pd
# Cartesian product of 2 iterables
from itertools import product
# Path name pattern searching
import glob
# OS interfaces
import os
# Compression interface
import zlib
# Binary encoding
from base64 import urlsafe_b64encode as b64e, urlsafe_b64decode as b64d
# Some utility functions, you can take a look into these if you're interested in how the code works
from src.utils import show,draw_orientations,gabor_kernel,draw_minutiae,angle_mean,angle_abs_difference
# The fingerprint enhancement pipeline
from src.fingerprint_enhancer import FingerprintImageEnhancer

import math

# Your imports here (if any)
# import ...


In [ ]:
# Helper functions
def plot_image_sequence(data, n, imgs_per_row=7, figsize=(10,10), cmap='gray'):
    n_rows = 1 + int(n/(imgs_per_row+1))
    n_cols = min(imgs_per_row, n)

    f,ax = plt.subplots(n_rows,n_cols, figsize=(figsize[0]*n_cols,figsize[1]*n_rows))
    for i in range(n):
        if n == 1:
            ax.imshow(data[i], cmap=cmap)
        elif n_rows > 1:
            ax[int(i/imgs_per_row),int(i%imgs_per_row)].imshow(data[i], cmap=cmap)
        else:
            ax[int(i%n)].imshow(data[i], cmap=cmap)
    plt.show()

## <a id='I.-Setting'></a>I. Setting

Oh no! A woman is found dead in her hotel room. Multiple stab wounds in the chest indicate that it was... *MURDER!*

The police has locked down the hotel and you were brought in to assist the forensics team in finding the murderer. Your associates have reviewed the security footage and have detected a person entering the room with the victim just 15 minutes before the estimated time of death. Unfortunately, the perpetrator did their homework and they were very careful not to give away any leads that might help you identify them. They slipped however, and on their way out touched the door knob with their bare hand...

## <a id='II.-Fingerprint-Recognition'></a> II. Fingerprint Recognition

### 1. Reading data
#### *Reading the image data and converting to grayscale*


Before you get started on anything it is always a good idea to visualise your data. Let's first have a look at the fingerprint that was collected from the door knob first...

Note that we convert the data to grayscale, we're not interested in color values (and don't have them).

In [ ]:
perpetrator_fp = cv2.imread('./perpetrator_fp.png',cv2.IMREAD_GRAYSCALE)


show((perpetrator_fp, "Fingerprint of the horrible person"))


Now, let's have a look at the other fingerprint data that was provided to you. This data is stores in `data/NIST301/`. There were apparently 100 guests in the hotel at the time of murder. The filename indicates the subject identifier. Below we have provided a function to read and label these fingerprints. Let's take a look!

In [ ]:
def read_DB(path):
    images = []
    labels = []
    imagePaths = sorted(Path(path).rglob("*.png"))
    for imagePath in imagePaths:
        image = cv2.imread(path + imagePath.name, cv2.IMREAD_GRAYSCALE)
        if (len(image.shape) > 2):
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        images.append(image)
        label = imagePath.stem[0:3]
        labels.append(label)
    return (images, labels)


# Read the fingerprint database
images_db, labels_db = read_DB('./data/NIST301/')

# Save some metadata
n_imgs = len(images_db)
img_height, img_width = images_db[0].shape

Markdown(f"""Here are some samples from the database, which contains {n_imgs} images of size {img_height}x{img_width}. Remember, visualization is always a good idea, so let's visualize some""")

In [ ]:
n = 14
plot_image_sequence(images_db,n)

### 2. A baseline model

#### 2.1. Choosing a similarity metric
In your first assignment the similarity metric was already provided, in this assignment you have the chance to play around with distances and similarities yourself! Let's give it a shot!

Consider the similarity function below:

In [ ]:
mss = lambda x,y: 1/(1+np.square(x-y).mean())

#### 2.2. Constructing the similarity score table
Since we're not in an authentication scenario, all we can do is to calculate the similarity scores between each fingerprint in the database and the one from the perpetrator.
Once we have a metric for each image, we can just sort them and store them in a table. Note that you have space to get creative with visualization!

In [ ]:
def constructSimilarityTable(org_img,img_db, labels, dist_func):
    #dist_func is the function that computes the distance between two images
    data=[]
    for i,img in enumerate(tqdm_notebook(img_db)):
        data.append([
            labels[i],
            dist_func(org_img, img)])
    assert (len(data) == len(img_db))
    return pd.DataFrame(data, columns=['id', 'score'])

sim_tb0 = constructSimilarityTable(perpetrator_fp,images_db, labels_db, mss)

In [ ]:
ids,scores = sim_tb0.sort_values(by='score', ascending=False).values[:,0],sim_tb0.sort_values(by='score', ascending=False).values[:,1]
plt.plot(ids,scores)
plt.xticks(np.arange(0,100,5),ids[np.arange(0,100,5)],rotation = 45)
plt.title('Highest Score ID: '+ids[0]);

Markdown(f'''We can see that the highest match score {scores[0]:.4f}
belongs to subject '''+ids[0]+'. However, subject '+ids[1]+f''' is not far behind with
a score of {scores[1]:.4f}! Surely we can do a better job than that. Or can we..?''')

2.3. Geometric sensitivity test of MSS

The suspect's fingerprint "perpetrator_fp" is rotated + size standardized to detect the sensitivity of MSS to geometric changes.

In [ ]:
def rotate_image(img, angle):
    """Keep the original size rotation"""
    h, w = img.shape
    center = (w//2, h//2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    # Calculate new boundaries to ensure no cropping
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))
    M[0, 2] += (new_w / 2) - center[0]
    M[1, 2] += (new_h / 2) - center[1]
    return cv2.warpAffine(img, M, (new_w, new_h))

In [ ]:
def resize_to_match(img, target_shape=(480, 320)):
    """"Scale the image to the target size"""
    return cv2.resize(img, (target_shape[1], target_shape[0]))

#  Test rotation invariance
rotated = rotate_image(perpetrator_fp, 90)  # Rotate 90 degrees
rotated = resize_to_match(rotated)  # Uniform size

# Caculate MSS
mss_score = mss(perpetrator_fp, rotated)
print(f"MSS score after rotation 90 degrees: {mss_score:.4f}")

In [ ]:
angles = [0, 30, 45, 90, 180]
results = []
for angle in angles:
    rotated = rotate_image(perpetrator_fp, angle)
    rotated = resize_to_match(rotated)
    score = mss(perpetrator_fp, rotated)
    results.append((angle, score))
    if angle == 90:  # Special record of 90 degree rotation score
        print(f"MSS score after rotation 90 degrees: {score:.4f}")

# Visualize results
df = pd.DataFrame(results, columns=['Angle', 'MSS Score'])
plt.plot(df['Angle'], df['MSS Score'], marker='o')

# Add 90-degree rotation marking points
plt.annotate(f'90°: {df[df["Angle"]==90]["MSS Score"].values[0]:.4f}',
             xy=(90, df[df["Angle"]==90]["MSS Score"]),
             xytext=(20, -20),
             textcoords='offset points',
             arrowprops=dict(arrowstyle="->"))

plt.xlabel('Rotation Angle (degrees)')
plt.ylabel('MSS Score')
plt.title('Rotation Sensitivity Test\n(MSS score after 90° rotation: 0.0128)')  # 在标题中再次强调
plt.grid(True)
plt.show()

<div class="alert alert-block alert-success">
<b>Q1: </b> Now that we have some results, is the given similarity function a good metric to quantify a distance between two fingerprints? Is it reliable enough to incriminate a suspect? What are its limitations?
</div>

### 3. Minutiae-Based Matching

We'll be using the tried-and-true method of minutiae detection in order to extract usable features from the images. This will help us have data that is consistent across images.

#### 3.1. Image Enhancement
In order to make life easy on ourselves in this investigation, we will first start by enhancing the fingerprint images.
First thing we should focus on is getting crisp ridge images. In order to do that we have to perform a couple of steps:

1. Segment the part of the image that actually contains the fingerprint
2. Estimate the ridge orientations
3. Estimate the local ridge frequencies

To perform these steps, we will be using a mix of code provided by [Utkarsh Deshmukh](https://github.com/Utkarsh-Deshmukh/Fingerprint-Enhancement-Python) and [BioLab Univerity of Bologna](https://colab.research.google.com/drive/1u5X8Vg9nXWPEDFFtUwbkdbQxBh4hba_M#scrollTo=LSe9qJ63zw_E).
Let's start with the segmentation. This step is fairly straight-forward. The foreground (fingerprint) contains many edges and the background does not. So, we will use a very simple technique based on the magnitude of the local gradient.

In [ ]:
# Calculate the local gradient (using Sobel filters)
fe = FingerprintImageEnhancer()

def get_mask(image):
    normalized_img,mask = fe.ridge_segment(image)
    return normalized_img,mask

normalized_img,mask = get_mask(perpetrator_fp)
masked_fp = perpetrator_fp * mask
show((perpetrator_fp, 'Original'), (masked_fp, 'Masked'))


Now moving onto estimating the local ridge orientations. This step is essentially finding the angle the fingerprint ridges form with the horizontal axis in a small neighborhood.

For each pixel, we will estimate the local orientation from the gradient $[Gx,Gy]$, which we already computed in the segmentation step (see *A.M. Bazen and S.H. Gerez, "Systematic methods for the computation of the directional fields and singular points of fingerprints," in IEEE tPAMI, July 2002*).

The ridge orientation is estimated as ortoghonal to the gradient orientation, averaged over a window $W$.  

$G_{xx}=\sum_W{G_x^2}$, $G_{yy}=\sum_W{G_y^2}$, $G_{xy}=\sum_W{G_xG_y}$

$\theta=\frac{\pi}{2} + \frac{phase(G_{xx}-G_{yy}, 2G_{xy})}{2}$



In [ ]:

def get_orientations():
    orientations = fe.ridge_orient()
    return orientations

orientations = get_orientations()
show(draw_orientations(perpetrator_fp, orientations/(-np.pi/3), np.ones_like(orientations)*0.5, mask, 1, 18), 'Orientation image')


We can see that the orientatitons are generally pointing in the right directions, even though they dont overlap with the ridges exactly. This is because the orientations are actually a vector field, and we just sample from this field.

Now all we need is to calculate the ridge frequency. We define the local ridge frequency as the number of ridges per unit length along a hypothetical small window. Initially, we will just assume constant frequency over the entire image and estimate the ridge period on within a set region. Then we reciprocate to get the frequency.

We do this by using the *x-signature* of the region.

*L. Hong, Y. Wan and A. Jain, "Fingerprint image enhancement: algorithm and performance evaluation," in IEEE tPAMI, Aug. 1998*

In [ ]:
region = perpetrator_fp[310:390,180:230]
smoothed = cv2.blur(region, (5,5), -1)
xs = np.sum(smoothed, 1) # the x-signature of the reg
x = np.arange(region.shape[0])
local_maxima = np.nonzero(np.r_[False, xs[1:] > xs[:-1]] & np.r_[xs[:-1] >= xs[1:], False])[0]
f, axarr = plt.subplots(1,2, sharey = True)
axarr[0].imshow(region,cmap='gray')
axarr[1].plot(xs, x)
axarr[1].set_ylim(region.shape[0]-1,0)
axarr[1].set_xticks(local_maxima)
axarr[1].grid(True, axis='y')
plt.show()

This sample shows how the ridge frequency is estimated. We could stop here, but notice how this process will yield different frequencies for each region we select. That is something we do not want. Instead, we should convolve over the image and estimate all local ridge frequencies to create a field that we can later sample for each pixel. We can once again use the provided code for this.

In [ ]:
def get_ridge_period(image):

    freq = fe.ridge_freq()
    return freq

ridge_freq = get_ridge_period(perpetrator_fp) # Using the transpose for presentation purposes


In [ ]:
show(255*(ridge_freq/ridge_freq.max()))

In [ ]:
Markdown(f'''The mean ridge frequency is {fe._mean_freq:.2f}. This means there is a ridge every {1/(fe._mean_freq):.2f} pixels over all the image.
         Note how the output mostly corresponds to the mask. This shows how the frequencies are gathered on the fingerprint ridges, unsurprisingly.''')


Now that we have all we need to enhance our images, we can start off by creating a Gabor filter bank that will accentuate the details of the fingerprint.

This step is actually a contextual convolution, where a different filter will be used for each pixel, based on the orientation value at the given point. By rotating the Gabor filters in accordance with the orientations we estimated, we make sure they capture the ridge.

In [ ]:
def gabor_filtering(image):
    ridge_im, gabor_filter = fe.ridge_filter()
    return gabor_filter,ridge_im

gabor_bank, enhanced = gabor_filtering(perpetrator_fp)


show(perpetrator_fp, enhanced)



In [ ]:
show(*gabor_bank[0::5])
show(*[cv2.filter2D(perpetrator_fp, cv2.CV_32F, gb) for gb in gabor_bank[0::5]])

We now apply the enhancement pipeline to all images.

In [ ]:
# Calcuate the enhanced images and the associated segmentation masks

def enhance_images(images):
    images_e_u = []
    masks = []
    orientations = []
    for i, image in enumerate(tqdm_notebook(images)):
        try:
            images_e_u.append(fe.enhance(image))
            masks.append(fe._mask)
        except:
            print('error for: ', i)
    return np.array(images_e_u), np.array(masks)

try:
    with open('enhanced_images.pkl', 'rb') as f:
        images_enhanced_db,masks_db= pickle.load(f)
except:
    images_enhanced_db, masks_db = enhance_images(images_db)
    with open('enhanced_images.pkl', 'wb') as f:
        pickle.dump((images_enhanced_db, masks_db), f)

images_enhanced_db, masks_db = enhance_images(images_db)

<div class="alert alert-block alert-info">
<b>Tip:</b> Intermediate (computation heavy) results can be saved on file using the pickle package, use <code>pickle.dump</code> to save and <code>pickle.load</code> to load data from a file. The code snippet below will run the computation and save results if there are no previous records, and reload them in later runs.
Have a look at the <a href="https://wiki.python.org/moin/UsingPickle">wiki</a> for more information.
<pre>
<code>
<!-- language: lang-python -->
try:
    with open("images_masks_db.pkl","rb") as f:
        images_enhanced_db, masks_db = pickle.load(f)
except:
    images_enhanced_db, masks_db = enhance_images(images_db)
    with open("images_masks_db.pkl",'wb') as f:
        pickle.dump([images_enhanced_db,masks_db],f)
</code>
</pre>

</div>


In [ ]:
 with open("images_masks_db.pkl","rb") as f:
        images_enhanced_db, masks_db = pickle.load(f)

In [ ]:
n=4
images_segmented_db = [a*b for a,b in zip(images_enhanced_db,masks_db)]
plot_image_sequence(images_db[:n] + images_enhanced_db.tolist()[:n] + masks_db.tolist()[:n], 3*n, n)

<div class="alert alert-block alert-success">
<b>Q2: </b> Take a look at the enhanced images. Do you see any challenges that you can face working with these?
</div>

#### 3.2. Minutiae Detection

Now that we have a clear image without noise, we can proceed to actually detect minutiae positions and directions. Starting off by generating a skeleton (not the spooky kind) where each line is one pixel wide, we will use a simple method of detecting minutiae.

In [ ]:
def skeletonize(image):
    try:
        return cv2.ximgproc.thinning(image,thinningType=cv2.ximgproc.THINNING_GUOHALL)
    except:
        image = 255*(image).astype(np.uint8)
        return cv2.ximgproc.thinning(image,thinningType=cv2.ximgproc.THINNING_GUOHALL)
skeleton = skeletonize(enhanced)
show(skeleton,"Skeleton")

To write our algorithm, we need to determine the criteria for terminations and bifurcations. We can do this in the following way:

-   For each pixel in the skeleton, we examine its immediate neighborhood of 8-pixels.
-   By definition, a termination pixel must have only one neighboring white pixel. So we look for white pixels with one white neighbor for terminations.
-   Bifurcations on the other hand, must have one pixel on the tail, and two on the fork. So we look for white pixels with three white neighbors.

<div class="alert alert-block alert-success">
<b>Q3: </b> Write your detection algorithm below. Keep in mind when you're inspecting the 8-neighborhood centered on a white pixel, the area also includes the pixel you're inspecting. Don't forget to update the criteria with this in mind!

Explain in your report how your algorithm works and show some example(s).
</div>

In [ ]:
def get_all_minutiae(skeleton):
    '''
    Generate minutiae from the skeleton image.
    Return a list of tuples: (x-coordinate, y-coordinate, True if termination False if bifurcation))
    '''
    minutiae = []

    for y in range(1, skeleton.shape[0] - 1):
        for x in range(1, skeleton.shape[1] - 1):
            if skeleton[y, x] == 255:  # white pixel
                # Take 8 neighbors around (note there are 9 pixels in total)
                neighbors = [
                    skeleton[y-1, x-1], skeleton[y-1, x], skeleton[y-1, x+1],
                    skeleton[y, x-1],                     skeleton[y, x+1],
                    skeleton[y+1, x-1], skeleton[y+1, x], skeleton[y+1, x+1]
                ]
                count = sum(1 for n in neighbors if n == 255)
                if count == 1:
                    minutiae.append((x, y, True))  # Termination
                elif count == 3:
                    minutiae.append((x, y, False))  # Bifurcation

    return minutiae

minutiae = get_all_minutiae(skeleton)
print(f"The total number of minuate detected: {len(minutiae)}")

minutiae = get_all_minutiae(skeleton)
show(draw_minutiae(skeleton,minutiae), "Minutiae")

There will be a lot of false detections with the above step on the borders of the fingerprint. We can get rid of these by applying an eroded mask, such that the minutiae too close to the mask border are excluded.

In [ ]:
def border_reduce(image,minutiae):
    gx, gy = cv2.Sobel(image, cv2.CV_32F, 1, 0), cv2.Sobel(image, cv2.CV_32F, 0, 1)
    gx2, gy2 = gx**2, gy**2
    gm = np.sqrt(gx2 + gy2)
    sum_gm = cv2.boxFilter(gm, -1, (25, 25), normalize = False)
    thr = sum_gm.max() * 0.2
    mask = cv2.threshold(sum_gm, thr, 255, cv2.THRESH_BINARY)[1].astype(np.uint8)
    mask_distance = cv2.distanceTransform(cv2.copyMakeBorder(255*(mask).astype(np.uint8), 1, 1, 1, 1, cv2.BORDER_CONSTANT), cv2.DIST_L1, 3)[1:-1,1:-1]
    filtered_minutiae = list(filter(lambda m: mask_distance[m[1], m[0]]>12, minutiae))
    return mask_distance,filtered_minutiae

mask_distance,filtered_minutiae = border_reduce(perpetrator_fp,minutiae)

show((mask,"Mask"), (mask_distance,"Transform"),(draw_minutiae(skeleton, filtered_minutiae),"Filtering Result"))


To estimate the directions of the minutiae, we can do a couple of things. We can simply sample the orientation map we generated at minutiae locations.   However, this would mean we assume that all minutiae we detected, and that is most likely not the case. Even with all the preprocessing and enhancements we made, there are likely to be some artifactual points, such as very short lines, or just noisy areas.

To remedy this, we need an additional filtering step. So we propose following rules.

- For terminations, the ridge must continue for 20 pixels from the minutia point, uninterrupted.  
- For bifurcations, each arm stemming from the minutia must adhere to the termination rule.
- If another minutia is detected within 10 steps of following the ridge, the original minutia is invalid.

This filtering method can also be overloaded in order to acquire the directions of the minutiae! By calculating the angle between our starting point, and the point we land on, we can find the direction of a termination. For bifurcations, we will just get the average angle of the two closest angles that we find.

<p>
    <img src="https://biolab.csr.unibo.it/samples/fr/images/min_directions.png">
</p>
<p>
    <em> Image from BioLab, University of Bologna </em>
</p>

<div class="alert alert-block alert-info">
This is a fairly complex process to code, so we provide you with the functions you will need. Feel free to dig in and explore!

Usage of the provided code:
<pre>
<code>
mf = MinutiaeFilter() # You need to do this just once, not for all images
valid_minutiae_filtered = mf.get_filtered(enhanced_image) # Using the enhanced image, not the original
</code>
</pre>

</div>

In [ ]:
class MinutiaeFilter:
    cn_filter = np.array([[  1,  2,  4],
                    [128,  0,  8],
                    [ 64, 32, 16]
                    ])
    def __init__(self):
        pass

    def __compute_crossing_number(self,values):
        return np.count_nonzero(values < np.roll(values, -1))

    def __set_stage(self,skel):
        self.all_8_neighborhoods = [np.array([int(d) for d in f'{x:08b}'])[::-1] for x in range(256)]
        self.cn_lut = np.array([self.__compute_crossing_number(x) for x in self.all_8_neighborhoods]).astype(np.uint8)

        skel01 = np.where(skel!=0, 1, 0).astype(np.uint8)
        # Apply the filter to encode the 8-neighborhood of each pixel into a byte [0,255]
        self.neighborhood_values = cv2.filter2D(skel01, -1, MinutiaeFilter.cn_filter, borderType = cv2.BORDER_CONSTANT)
        self.cn = cv2.LUT(self.neighborhood_values, self.cn_lut)
        # Keep only crossing numbers on the skel
        self.cn[skel==0] = 0

        r2 = 2**0.5 # sqrt(2)
        # The eight possible (x, y) offsets with each corresponding Euclidean distance
        self.xy_steps = [(-1,-1,r2),( 0,-1,1),( 1,-1,r2),( 1, 0,1),( 1, 1,r2),( 0, 1,1),(-1, 1,r2),(-1, 0,1)]
        # LUT: for each 8-neighborhood and each previous direction [0,8],
        #      where 8 means "none", provides the list of possible directions
        self.nd_lut = [[self.compute_next_ridge_following_directions(pd, x) for pd in range(9)] for x in self.all_8_neighborhoods]


    def get_filtered(self,image_enhanced):

        skel = skeletonize(image_enhanced)

        self.__set_stage(skel)
        minutiae = get_all_minutiae(skel)
        try:
            _,filtered_minutiae = border_reduce(image_enhanced,minutiae)
        except:
            _,filtered_minutiae = border_reduce(255*image_enhanced.astype(np.uint8),minutiae)
        valid_minutiae = []
        for x, y, term in filtered_minutiae:
            d = None
            if term: # termination: simply follow and compute the direction
                d = self.follow_ridge_and_compute_angle(x, y)
            else: # bifurcation: follow each of the three branches
                dirs = self.nd_lut[self.neighborhood_values[y,x]][8] # 8 means: no previous direction
                if len(dirs)==3: # only if there are exactly three branches
                    angles = [self.follow_ridge_and_compute_angle(x+self.xy_steps[d][0], y+self.xy_steps[d][1], d) for d in dirs]
                    if all(a is not None for a in angles):
                        a1, a2 = min(((angles[i], angles[(i+1)%3]) for i in range(3)), key=lambda t: angle_abs_difference(t[0], t[1]))
                        d = angle_mean(a1, a2)
            if d is not None:
                valid_minutiae.append( (x, y, term, d) )

        return valid_minutiae

    def compute_next_ridge_following_directions(self,previous_direction, values):
        next_positions = np.argwhere(values!=0).ravel().tolist()
        if len(next_positions) > 0 and previous_direction != 8:
            # There is a previous direction: return all the next directions, sorted according to the distance from it,
            #                                except the direction, if any, that corresponds to the previous position
            next_positions.sort(key = lambda d: 4 - abs(abs(d - previous_direction) - 4))
            if next_positions[-1] == (previous_direction + 4) % 8: # the direction of the previous position is the opposite one
                next_positions = next_positions[:-1] # removes it
        return next_positions

    def follow_ridge_and_compute_angle(self,x, y, d = 8):
        px, py = x, y
        length = 0.0
        while length < 20: # max length followed
            next_directions = self.nd_lut[self.neighborhood_values[py,px]][d]
            if len(next_directions) == 0:
                break
            # Need to check ALL possible next directions
            if (any(self.cn[py + self.xy_steps[nd][1], px + self.xy_steps[nd][0]] != 2 for nd in next_directions)):
                break # another minutia found: we stop here
            # Only the first direction has to be followed
            d = next_directions[0]
            ox, oy, l = self.xy_steps[d]
            px += ox ; py += oy ; length += l
        # check if the minimum length for a valid direction has been reached
        return math.atan2(-py+y, px-x) if length >= 10 else None


<div class="alert alert-block alert-success">
<b>Q4: </b> Comment on the advantages of detecting the orientations of minutiae on top of just locations.
</div>

In [ ]:
mf = MinutiaeFilter()
valid_minutiae = mf.get_filtered(enhanced)

show(draw_minutiae(skeleton,filtered_minutiae),draw_minutiae(skeleton,valid_minutiae),draw_minutiae(perpetrator_fp, valid_minutiae))

Whew, those were some long steps to get robust minutiae! But now that we have a method of generating them, we need to do come up with a way to match these, to find our perpetrator and bring him to justice!

#### 3.3. Global Matching and Image Alignment

Here we use the matching keypoints to align the fingerprints, this allows us to compare the images as a whole (globally). We will first align our images with respect to the local descriptors around our minutiae, then we will compare the geometric distances between the matching sets to compute similarity.

[Here](https://www.learnopencv.com/image-alignment-feature-based-using-opencv-c-python/) you can find a description and code how to start from the brute force matching results and estimate the best transformation (from a family of transformations) that aligns the two images. In the example, a homography-type transformation is searched for. However, this has too many degrees of freedom for our application. We substituted this by a more constrained (only 4 degrees of freedom) similarity (partial affine) transformation.

These routines iteratively determine the minimal set of matching points that define a transformation that optimally aligns all other points as well, taking care of outliers at the same time. This method is a very general optimization technique and is called RANSAC, for "RANdom SAmple Consensus". See, apart from many other sources on the internet, [this presentation](http://www.cse.psu.edu/~rtc12/CSE486/lecture15.pdf) for further explanation.

In order to use our minutiae with the OpenCV matching pipeline, we have to pull a fast one on OpenCV. We do a cheeky KeyPoint conversion with [<code>cv2.KeyPoint</code>](https://docs.opencv.org/3.4/d2/d29/classcv_1_1KeyPoint.html), set the coordinates to those of our minutiae, and use our minutiae as just as we would use <code>cv2.KeyPoint</code>.


In [ ]:
def minutia2kp(minutiae):
    return [cv2.KeyPoint(x, y, 10, angle, 0, 0, -1) for x, y, _, angle in minutiae] #no angle or type for the keypoints...

![hehe](./img/meme.png)

Now we can feed the minutiae to the detectors to get our local descriptors, and ultimately use these descriptors for image alignment.

In [ ]:
testNr1 = 6
testNr2 = 18

def compute_local_descriptor(img, minutiae):
    detector = cv2.ORB_create()
    kp = minutia2kp(minutiae)
    kp, des = detector.compute(img, kp)
    return np.array(kp), des

mn1 = mf.get_filtered(images_enhanced_db[testNr1])
mn2 = mf.get_filtered(images_enhanced_db[testNr2])


In [ ]:
mn1 , local_des1 = compute_local_descriptor(images_enhanced_db[testNr1],mn1 )
mn2 ,local_des2 = compute_local_descriptor(images_enhanced_db[testNr2],mn2 )


**The Local Feature descriptor** will give you a vector describing the local region around the minutia in the feature space. Note that these feature descriptors give a vectorial summary of the neighbourhood around minutiae. A simple metric on these vectors (Euclidean Distance for continuous variables, Hamming Distance for binary variables) can then be used to determine similarity. These are what the matcher will compare when evaluating a pair of minutiae.

In [ ]:
def brute_force_matcher(des1, des2, dist=cv2.NORM_HAMMING):
    """
      Brute Force matcher on a pair of KeyPoint (hehe) sets using the local descriptor for similarity

      returns all pairs of best matches
    """

    # crossCheck=True only retains pairs of keypoints that are each other best matching pair
    bf = cv2.BFMatcher(dist, crossCheck=True)
    matches = list(bf.match(des1, des2))

    # sort matches based on descriptor distance
    matches.sort(key=lambda x: x.distance, reverse=False)

    return np.array(matches)

local_k1_k2_matches = brute_force_matcher(local_des1, local_des2,cv2.NORM_HAMMING)

local_pt_source, local_pt_target = np.array([
    (match.queryIdx, match.trainIdx) for match in local_k1_k2_matches]).T

def estimate_affine_transform_by_kps(src_pts, dst_pts):
    """
        Returns the Affine transformation that aligns two sets of points
    """
    transform_matrix, inliers = cv2.estimateAffinePartial2D(src_pts, dst_pts,
                                           method =  cv2.RANSAC,
                                           confidence = 0.9,
                                           ransacReprojThreshold = 10.0,
                                           maxIters = 5000,
                                           refineIters = 10)
    return transform_matrix, inliers[:,0]

def warp_points(pts, M):
    mat_reg_points = cv2.transform(pts.reshape(-1,1,2), M)

    # return transformed keypoint list
    return cv2.KeyPoint.convert(np.array(mat_reg_points).reshape(-1,2))


def warp_img(img, M):
    return cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))

# estimate the affine transform that aligns the matched keypoints
M, inliers = estimate_affine_transform_by_kps(
    cv2.KeyPoint.convert(np.array(mn1)[local_pt_source]),
    cv2.KeyPoint.convert(np.array(mn2)[local_pt_target]))

mn1_reg = warp_points(cv2.KeyPoint.convert(np.array(mn1)), M)
img1_reg =  warp_img(images_enhanced_db[testNr1], M)

**Point Matchers** match points that have similar descriptors in an image pair and computes the distances between the best matching pairs of keypoints. In this implementation we make use of the brute force matcher of OpenCV, documentation can be found [here](https://opencv24-python-tutorials.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_matcher/py_matcher.html). As a distance between descriptors we make use of the normalised [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance). Usually you will see these matchers used with keypoint detection algorithms such as SIFT, but in our case we have a more reliable way of locating points of interest!


Let's have a look at how the images are alinged. Try visualizing different pairs of fingerprints! You may notice the estimated transform will be more extreme for images that are less similar, and will become more subtle as the images become more similar.

In [ ]:
show_img1 = cv2.drawKeypoints(img1_reg, mn1_reg, None, (0, 255, 0), cv2.DRAW_MATCHES_FLAGS_DEFAULT)
show_img2 = cv2.drawKeypoints(images_enhanced_db[testNr2], mn2, None, (0, 255, 0), cv2.DRAW_MATCHES_FLAGS_DEFAULT)

show(show_img1, show_img2)

As you might have noticed the `estimateAffinePartial2D` function does not only return the transformation matrix but also the minimal set of matching points that were used to to determine the transformation. This gives us a better, more refined set of keypoints.

In [ ]:
global_k1_k2_matches = local_k1_k2_matches[inliers == 1]

imMatches = cv2.drawMatches(img1_reg, mn1_reg, images_enhanced_db[testNr2], mn2, global_k1_k2_matches.tolist(), None)
show(imMatches)

<div class="alert alert-block alert-success">
<b>Q5:</b> Are all the keypoint matches accurate? Are they expected to be? Explain why.
</div>

#### 3.4. Global Similarity Metric
Now that we have matching keypoints we can define simple scalar measures on this set of distances, such as the number of pairs with a distance smaller than a set threshold, or the sum/mean of the first N distances (ranked from small to larger), etc.

<div class="alert alert-block alert-success">
<b>Q6: </b> Choose a global feature similarity function, (e.g. you can start from euclidean distance between the reduced sets of KeyPoints and count the values above a threshold).
</div>

<div class="alert alert-block alert-info">
<b>Tip:</b> Make sure you spent the proper amount of time on constructing the similarity metrics, a good implementation will save you a lot of time in the long run! </div>

In [ ]:
def global_img_similarity(matches, reg_kp1, kp2):
    '''
    Given the matches, and more importantly the matching minutiae,
    return a similarity score between the two images.
    '''
    if len(matches) == 0:
        return 0  # no matches found

    # Extract matched keypoints
    src_pts = np.array([reg_kp1[m.queryIdx].pt for m in matches])
    dst_pts = np.array([kp2[m.trainIdx].pt for m in matches])

    # Compute Euclidean distances between matched keypoints
    distances = np.linalg.norm(src_pts - dst_pts, axis=1)

    # Threshold to consider a match as "good"
    threshold = 10.0  # you can tune this value if needed

    # Count how many distances are below the threshold
    good_matches = np.sum(distances < threshold)

    # Normalize by total number of matches (optional, depending on whether you want a score between 0 and 1)
    sim_score = good_matches / len(matches)

    return sim_score

In [ ]:
perp_img_en,perp_img_mask = 255*enhanced.astype(np.uint8), mask

def global_similarity(masked_fp1, masked_fp2, detector=cv2.ORB_create(), kp_erosion_ksize = (5,5)):
    # separate the semgentation from the image
    fp1, mask1 = masked_fp1[...,0], masked_fp1[...,1]
    fp2, mask2 = masked_fp2[...,0], masked_fp2[...,1]


    # detect the keypoints
    mn1 = mf.get_filtered(fp1)
    mn2 = mf.get_filtered(fp2)

    # compute descriptor for each keypoints

    mn1, local_des1 = compute_local_descriptor(fp1, mn1)
    mn2, local_des2 = compute_local_descriptor(fp2, mn2)

    # find matches between keypoints based on local feature descriptor
    local_k1_k2_matches = brute_force_matcher(local_des1, local_des2, cv2.NORM_HAMMING)

    # get source and target index for each match
    local_pt_source, local_pt_target = np.array([
        (match.queryIdx, match.trainIdx) for match in local_k1_k2_matches]).T

    # use matching keypoints to estimate an affine transform between keypoints
    M, inliers = estimate_affine_transform_by_kps(
        cv2.KeyPoint.convert(mn1[local_pt_source]),
        cv2.KeyPoint.convert(mn2[local_pt_target]))

    # if no inliers can be found
    if M is None:
        return 0

    # warp the keypoints according to the found transform
    mn1_reg = warp_points(cv2.KeyPoint.convert(mn1), M)

    # subset the keypoints, inliers are considered good keypoints
    # since they were used in finding the transformation
    global_k1_k2_matches = local_k1_k2_matches[inliers == 1]

    # compute global similarity based aligned matching global keypoints
    return global_img_similarity(global_k1_k2_matches,mn1_reg, mn2)


sim_tb1 = constructSimilarityTable(np.stack((perp_img_en,perp_img_mask),-1), np.stack((images_segmented_db,masks_db),-1), labels_db, global_similarity)

#### 3.5. Validation
<div class="alert alert-block alert-success">
<b>Q7: </b> Visualize the scores and determine a score threshold to discriminate the matching fingerprints. Explain how you determine the threshold.
</div>

In [ ]:
def global_similarity(masked_fp1, masked_fp2, detector=cv2.ORB_create(), kp_erosion_ksize=(5,5)):
    # separate the segmentation from the image
    fp1, mask1 = masked_fp1[..., 0], masked_fp1[..., 1]
    fp2, mask2 = masked_fp2[..., 0], masked_fp2[..., 1]

    # detect the keypoints
    mn1 = mf.get_filtered(fp1)
    mn2 = mf.get_filtered(fp2)

    # compute descriptor for each keypoint
    mn1, local_des1 = compute_local_descriptor(fp1, mn1)
    mn2, local_des2 = compute_local_descriptor(fp2, mn2)

    # find matches between keypoints based on local feature descriptor
    local_k1_k2_matches = brute_force_matcher(local_des1, local_des2, cv2.NORM_HAMMING)

    # get source and target index for each match
    local_pt_source, local_pt_target = np.array([
        (match.queryIdx, match.trainIdx) for match in local_k1_k2_matches]).T

    # use matching keypoints to estimate an affine transform between keypoints
    M, inliers = estimate_affine_transform_by_kps(
        cv2.KeyPoint.convert(mn1[local_pt_source]),
        cv2.KeyPoint.convert(mn2[local_pt_target]))

    # if no inliers can be found
    if M is None:
        return 0

    # warp the keypoints according to the found transform
    mn1_reg = warp_points(cv2.KeyPoint.convert(mn1), M)

    # subset the keypoints, inliers are considered good keypoints
    # since they were used in finding the transformation
    global_k1_k2_matches = local_k1_k2_matches[inliers == 1]

    # compute global similarity based aligned matching global keypoints
    return global_img_similarity(global_k1_k2_matches, mn1_reg, mn2)


A good idea in this plot, where we cannot pick out one clear subject (as the first 3 subjects are all pretty close), is to take a look at the knee point for a score threshold. Also, let's normalize our scores in order to get a well generalized scale.

In [ ]:
#Simple module to find the knee in a series
from kneed import KneeLocator

scores_local = (scores - scores.min()) / (scores.max() - scores.min())

kneedle = KneeLocator(list(range(len(ids))),scores_local,S=1, curve='convex', direction='decreasing')
print(f'Knee at: {kneedle.knee}\t Treshold: ',kneedle.knee_y)
kneedle.plot_knee()
Markdown(f" With the threshold of {kneedle.knee_y:.3f}, we can see a more clear divide between the subjects. However, the best {kneedle.knee} matches are possible suspects... " )

No clear distinction... This is suspicious, let's take a look at the best matching fingerprints.

In [ ]:
def get_image_by_label(ind):
    for i in range(len(labels_db)):
        if labels_db[i] == ind:
            return images_enhanced_db[i]
    raise ValueError("No image with this label is found.")

# sim_tb1 should be sorted by score for this cell
imgs = [get_image_by_label(i) for i in sim_tb1.values[:10,0]]

plot_image_sequence(imgs,10)

They are all the same fingerprint! The murderer must have hacked the database and replicated some of the images, effectively covering their tracks. This must be an inside job! They seem to be one step ahead... Or so they think! Fingerprints are not the only biometric you can evaluate.

Since you can't trust your immediate co-workers anymore (afterall whoever did this had access to your database), you call your friends at CSI New York and send them the video surveillance records. By making use of the revolutionary (and non-existent) technology of ***zoom-and-enhance***, they return to you a database of iris images.

![Zoom and Enhance](./img/csi_zoom_enhance.gif)

***The plot thickens...***

## III. Iris recognition
The investigation is back on track! Now let's create a biometric system for the iris data.

### 1. Reading data

<div class="alert alert-block alert-success">
<b>Q8: </b> Check out <code>iris_perpetrator.png</code>. Where do you see difficulties? What kind of similarity measures do you expect to work best?
</div>

In [ ]:
iris_perpetrator = cv2.cvtColor(cv2.imread("perpetrator_iris.bmp"),cv2.COLOR_BGR2GRAY)
show((iris_perpetrator,"Look at this cold, dead eye..."));

First things first, let's load the iris images and inspect a few examples.

In [ ]:
# set path
iris_data_path = 'data/CASIA1'
def read_iris_DB(path):
    images = []
    labels = []

    for dirpath, _, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith(".jpg"):
                label = int(dirpath.split('/')[-1])
                im = cv2.imread(os.path.join(dirpath,filename),cv2.IMREAD_GRAYSCALE)
                labels.append(label)
                images.append(im)
    return np.array(images), np.array(labels)


# read iris Database
iris_images,iris_labels = read_iris_DB(iris_data_path)

# shuffle the data
idc = np.arange(len(iris_images))
np.random.shuffle(idc)
iris_images, iris_labels = iris_images[idc], iris_labels[idc]

In [ ]:
n = 7
plot_image_sequence(iris_images[10:20], n)

### 2. Biometric iris Recognition System
#### 2.1 Image enhancement

We can see the raw images may not be the best for further identification tasks, so we we will segment the irises and preprocess the images. To do this, we will make use of the code by. These modules will help us in the next steps.

We can segment the iris out of the image with several steps:
1. Edge detection: We can use edge detection to localize the borders. in out image.
2. [Hough transform](https://www.sciencedirect.com/topics/computer-science/hough-transforms#:~:text=The%20Hough%20transform%20(HT)%20%5B,the%20Radon%20transform%20%5BDeans81%5D.): This transform will help us detect the round shape of the iris. This way we will be able to select only the region of interest.
3. [Daugman Normalization](https://ijsta.com/papers/ijstav1n1/IJSTA_V1N1P3_PP11-14.pdf): We'll use this method to "unroll" our iris images to get a nice rectangular segment.

P.S. For a more detailed demonstration of these steps, you can refer to this repo by [Sobhan Shukueian](https://github.com/sobhanshukueian/Iris-Identification/blob/main/preprocess.ipynb).

<div class="alert alert-block alert-info">
<b>Tip: </b> The below cell is a lenghthy calculation (~20 min)! If you want adjust parameters or fiddle with the code, it is recommended you do it on a small sample of a dataset, and proceed once you have decided the best setup.
</div>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from src.irismodules.fnc import segment, normalize


# Feel free to play around with these!

eyelashes_thres = 80
radial_res = 200
angular_res = 500

def segment_iris(img):
    # Segment the iris region from the eye image. Indicate the noise region.
    ciriris, cirpupil, imwithnoise = segment.segment(img,eyelashes_thres=eyelashes_thres)

    # Normalize iris region by unwraping the circular region into a rectangular block of constant dimensions.
    polar_iris, mask = normalize.normalize(
        imwithnoise, ciriris[1], ciriris[0], ciriris[2], cirpupil[1], cirpupil[0], cirpupil[2], radial_res, angular_res)

    return polar_iris, (mask == 0)

def get_filter_bank(ksize = 5,sigma = 4, theta_range = np.arange(0,np.pi,np.pi/16), lambd=10,gamma = 0.5,psi=0 ):
    # this filterbank comes from https://cvtuts.wordpress.com/
    filters = []
    for theta in theta_range:
        kern = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)
        kern /= 1.5*kern.sum()
        filters.append(kern)
    return filters

def enhance_iris(img, eps = 1.e-15, agg_f = np.max):
    # get the gabor filters
    filters = get_filter_bank()

    # apply filters to image
    enhanced_image = np.array([cv2.filter2D(img, ddepth = -1, kernel=k) for k in filters])

    # Normalize in the [0,255] range
    enhanced_image = cv2.normalize(enhanced_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=0)

    # aggregate features
    return agg_f(enhanced_image,0)

def enhance_and_segment_irises(images):
    enhanced_images, masks = [], []
    for i,img in enumerate(tqdm_notebook(images)):
        try:
            normalised_img, mask = segment_iris(img)
            enhanced_img = enhance_iris(normalised_img)

            masks.append(mask)
            enhanced_images.append(enhanced_img)
        except:
            print(f"Problem with {i}, skipping")
            continue
    return np.array(enhanced_images), np.array(masks)

try:
    with open("iris_mask_db.pkl","rb") as f:
        enhanced_iris,mask_iris,iris_images,iris_labels = pickle.load(f)
except:
    enhanced_iris, mask_iris = enhance_and_segment_irises(iris_images)
    with open("iris_mask_db.pkl","wb") as f:
        pickle.dump([enhanced_iris,mask_iris,iris_images,iris_labels],f)



In [ ]:
n=5
iris_segmented_db = np.array([a*b for a,b in zip(enhanced_iris, mask_iris)])
plot_image_sequence(enhanced_iris.tolist()[:n] + mask_iris.tolist()[:n] + iris_segmented_db.tolist()[:n], 3*n, n, figsize= (10,5), cmap='gray')

<div class="alert alert-block alert-success">
<b>Q9: </b> Inspect the enhanced iris images. Do any of your difficulty predictions still hold? Do you see new ones?
</div>

#### 2.2. Triplet Loss Encoder
We're running out of options with regard to what other evidence to evaluate, so we do not want to take any more chances.

It's time to bring out the big guns. We will use a Triplet Encoder neural network to catch the murderer.

A triplet encoder is a specific type of architecture that has the goal of projecting inputs to a latent space, in which samples of the same class are closer together, and the embeddings of different classes are far apart.
We train these kinds of models by using triplets of (anchor, positive, negative). The anchor is the original image, the positive sample is a sample of the same class, and the negative sample is of a different class.

<img src="./img/triplet.webp" alt="Triplet Model" width="800"/>

The triplet loss function can be represented by the following equation:

$\mathcal{L}(a, p, n) = \max(0, \alpha + d(a, p) - d(a, n))$

Where:
- $ \mathcal{L}(a, p, n)$ is the triplet loss for the anchor $ a$, positive sample $ p$, and negative sample $ n$.
- $ \alpha$ is a margin hyperparameter that controls the minimum difference between the distances of positive and negative samples from the anchor. It has to be non-zero, and fairly large to prevent the network from cheating.
- $ d(a, p)$ is the distance between the anchor $ a$ and the positive sample $ p$ in the embedding space.
- $ d(a, n)$ is the distance between the anchor $ a$ and the negative sample $ n$ in the embedding space.

The triplet loss penalizes the model when the distance between the anchor and the positive sample is not smaller than the distance between the anchor and the negative sample by at least \( \alpha \). Otherwise, it incurs no loss. This encourages the model to learn embeddings where similar samples are closer together and dissimilar samples are farther apart. Read more about the triplet loss [here](https://medium.com/deep-learning-hk/compute-document-similarity-using-autoencoder-with-triplet-loss-eb7eb132eb38).

We will start off by coding a dataset and a dataloader. We'll be using [PyTorch](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html) for all our neural network needs. It has extensive documentation, so if you have a problem, you can likely find the answer there.

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T #for image transformations
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # no mps, it's very slow.
print(device)

In [ ]:

# Setting up random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

#split train and valudation sets

iris_val_images = np.zeros_like(iris_segmented_db[:len(np.unique(iris_labels))])
iris_val_labels = np.zeros_like(np.unique(iris_labels))
for i,l in enumerate(np.unique(iris_labels)):

    iris_val_images[i] = iris_segmented_db[iris_labels == l][0]
    iris_val_labels[i] = iris_labels[iris_labels == l][0]

    #Remove the validation images from the training set
    iris_segmented_db = np.delete(iris_segmented_db, np.argwhere(iris_labels == l)[0], axis = 0)
    iris_labels = np.delete(iris_labels, np.argwhere(iris_labels == l)[0])


#Create a Custom Dataset class

class IrisDataset(Dataset):
    def __init__(self, images, labels, transform=None,device = torch.device("cpu")):

        self.images = torch.tensor(images).unsqueeze(1).to(device)/255
        self.labels = torch.tensor(labels).to(device)
        self.transform = transform
        self.index = torch.arange(self.images.shape[0]).to(device)



    def __len__(self):
        return len(self.images)

    def get_random_positive(self,label):
        # To get random positive, we can't find one in the batch
        pos_idx = self.index[self.labels == label]
        idx = torch.randint(0,len(pos_idx),(1,))
        im = self.images[idx]
        return im

    def get_random_negative(self,label):
        # For random negative if there are no hard negatives in the batch
        neg_idx = self.index[self.labels != label]
        idx = torch.randint(0,len(neg_idx),(1,))
        im = self.images[idx]
        return im

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        return image, label


transform = T.Compose([
    T.Resize((100,200))
])

train_set = IrisDataset(iris_segmented_db, iris_labels, transform=transform, device = device)
val_set = IrisDataset(iris_val_images, iris_val_labels, transform=transform ,device = device)

batch_size = 256
iris_loader = DataLoader(train_set, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_set, batch_size=len(iris_val_images), shuffle=False)

batch_images, batch_labels = next(iter(iris_loader))

show(*batch_images[:5,0].cpu().numpy())


Now that we have a wrapper to get our data in the correct format, we can focus on the model and the training! We will use a convolutional encoder. and train it with a triplet loss.
Let's start by defining a model.

In [ ]:
class IrisEncoder(nn.Module):
    def __init__(self,embed_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1,16,kernel_size=4, stride=2, padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(),
            nn.Conv2d(16,32,kernel_size=4, stride=2, padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU()
        )
        self.linear = nn.Sequential(
            nn.Linear(32*12*31,embed_dim)
        )


    def forward(self,x):
        x = self.encoder(x)
        x = self.linear(x.view(-1,32*12*31))
        return x

Markdown("You can see we have two Conv layers with ReLU activations and max pooling. After this initial convolution block, we use a single fully connected layers to get the final embedding.")

One more thing before we can actually train our encoder; we need a [triplet mining](https://omoindrot.github.io/triplet-loss#triplet-loss-and-triplet-mining) strategy. We can do random triplet selection, which is fine in some cases, or we can be a little more refined and choose *semi-hard* triplets. This kind of triplet selection gives us a set where the negative sample is not closer to the anchor than to positive, but it still lies within the margin, such as:

$d(a, p) < d(a,n) < d(a,p) + \alpha$

You can think od this as specifically selecting negatives with which the network has trouble, so it can learn to differentiate them.


In [ ]:
def get_semihard_triplet_embeddings(batch,labels,model,dataset,device,margin):
        embeddings = model(batch.to(device))

        #Find the furthest positive for each sample
        triplets = torch.zeros(embeddings.shape[0],3,embeddings.shape[1]).to(device)

        for i in range(embeddings.shape[0]):
            label = labels[i].item()
            sample = embeddings[i]
            rel = embeddings[labels==label]
            pdist = 0
            max_d_positive = None
            triplets[i,0]=sample #the anchor
            #Look for a positive that is furthest from the anchor
            for positive in rel:
                d = torch.dist(sample,positive)
                if d > pdist:
                    max_d_positive=positive
                    pdist = d
            if max_d_positive is None:
                # find a random positive if no positive meeting the criteria is found
                im= dataset.get_random_positive(label)
                embed = model(im.to(device))
                max_d_positive = embed
            if max_d_positive is None:
                raise ValueError("No positives found.")
            triplets[i,1]=max_d_positive #the positive

            #Find semi-hard negative for each sample
            rel = embeddings[labels!=label]
            rel_label = labels[labels!=label]
            n_dist = 10000
            min_d_margin = None
            min_d_negative = None
            for j,negative in enumerate(rel):
                d=torch.dist(sample,negative)
                if d<n_dist:
                    if pdist<d<pdist+margin:
                        min_d_negative=negative
                        n_dist = d
                        min_d_margin=margin
            n_dist = 1000
            if min_d_negative is None:
                #If no semi-hard negative is found,go for hard negative (closer than the positive)
                for j,negative in enumerate(rel):
                    d=torch.dist(sample,negative)
                    if d<n_dist:
                        min_d_negative=negative
                        n_dist = d
                        min_d_margin=margin
            if min_d_negative is None:
                #if no hard negatives are found get, like, any negative?
                im= dataset.get_random_negative(label)
                embed = model(im.to(device))
                min_d_negative = embed
            if min_d_negative is None:
                raise ValueError("No negatives found.")
            triplets[i,2]=min_d_negative #the negative

        return triplets

In [ ]:
# Training setup
embed_dim = 64
margin = 0.7
model = IrisEncoder(embed_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

epochs = 60

Now we train the encoder model with all the data we have!

<div class="alert alert-block alert-info">
<b>Tip: </b> When it comes to neural net training, slow and steady usually wins the race. Use a low enough learning rate, and don't be discouraged if the loss doesn't change in the first ~30 epochs. We're trying to approximate a fairly complex function, so it may take a while to see results.
</div>

In [ ]:
def validate(model,loader):
    with torch.no_grad():
        for val_ims,val_labels in loader:
            embeddings = model(val_ims.to(device))
            sim = torch.cdist(embeddings,embeddings)
            sim = sim.mean()
        return sim


#training loop

best_model_wts = model.state_dict()
best_dst = 0
t= tqdm_notebook(range(epochs))
for epoch in t:
    running_loss = 0.0
    for i, data in enumerate(iris_loader):
        inputs, labels = data
        optimizer.zero_grad()
        triplets = get_semihard_triplet_embeddings(inputs,labels,model,iris_loader.dataset,device,margin=margin)

        anchor, positive, negative = triplets[:,0],triplets[:,1],triplets[:,2]
        l2_diff = ((anchor-positive)**2).sum(dim=1) - ((anchor-negative)**2).sum(dim=1)
        loss_triplet = torch.maximum((l2_diff+margin).mean(), torch.tensor(0))
        loss_triplet.backward()
        optimizer.step()
        running_loss += loss_triplet.item()
    val_distance = validate(model,val_loader)
    msg= f"Epoch {epoch+1}, Train Loss: {running_loss/len(iris_loader):.5f}, Mean Val Distance: {val_distance:.3f}"
    if val_distance > best_dst:
        best_dst = val_distance
        best_model_wts = model.state_dict()
        msg="*" + msg

    t.set_description(msg)


#### 2.3. Validation
Great! Now let's take a look at how well our network learned.

<div class="alert alert-block alert-info">
<b>Tip: </b> During training, you can use a validation set to track how well your model is doing on unseen data. This allows you to checkpoint your model based on the validation performance. You can use this snapshot of the model instead of the version at the end of training. The code to do this is below:
<pre>
<code>
<!-- language: lang-python -->
torch.save(best_model_wts,"best.ckpt") # to save on disk for later runs
model.load_state_dict(best_model_wts) # to load the weights of best validation score
</code>
</pre>
</div>

In [ ]:
torch.save(best_model_wts,"best.ckpt") # to save on disk for later runs
model.load_state_dict(best_model_wts) # to load the weights of best validation score

In [ ]:
# Get all embeddings
embs =[]
labs = []
for b,l in iris_loader:
    with torch.no_grad():
        embs.append(model(b.to(device)).cpu())
        labs.append(l)
embs = torch.vstack(embs)
labs = torch.cat(labs).flatten()

In [ ]:
dists = torch.zeros(len(labs.unique()),len(labs.unique()))
counts = torch.zeros(len(labs.unique()),len(labs.unique()))
for i,e in enumerate(embs):
    l1 = labs[i]-1
    for j,e2 in enumerate(embs):
        l2 = labs[j]-1
        dists[l1,l2]+=torch.dist(e,e2)
        counts[l1,l2]+=1

from mpl_toolkits.axes_grid1 import make_axes_locatable
ax = plt.subplot()
im = ax.imshow(((dists/counts).numpy()/(dists/counts).max()).numpy(),cmap = 'jet');
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im,cax=cax);

<div class="alert alert-block alert-success">
<b>Q10: </b> Visualize the embeddings in a lower-dimensional space (2D or 3D) using a dimensionality reduction method. Interpret the plot.
</div>


In [ ]:
from sklearn.manifold import TSNE

# Use t-SNE to reduce dimensionality to 2D
tsne = TSNE(n_components=2, random_state=42)
embeddings_2d_tsne = tsne.fit_transform(embs.numpy())

# Visualization
plt.figure(figsize=(8, 6))
scatter = plt.scatter(embeddings_2d_tsne[:, 0], embeddings_2d_tsne[:, 1], c=labs.numpy(), cmap='jet')
plt.colorbar(scatter)
plt.title("2D t-SNE of Embeddings")
plt.xlabel("t-SNE Component 1")
plt.ylabel("t-SNE Component 2")
plt.show()

As we expected, the model pushed the embeddings of different subjects apart in the latent space. Now all we have to do is to enhance and encode the perp image, then we can bring them to justice!

<div class="alert alert-block alert-success">
<b>Q11: </b> Ball's in your field. Enhance the perpetrator iris image, push it through the encoder you trained , and construct a similarity table just as you did with the fingerprints. Visualize your similarity scores and choose a threshold. Discuss results in your report.
</div>

<div class="alert alert-block alert-info">
<b>Tip:</b> Don't forget to apply the enhancement and masking to your perpetrator iris image.
</div>

In [ ]:
# Your code here...

In [ ]:
# Enhance criminal's iris image, process through encoder, build similarity table
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pandas as pd
from kneed import KneeLocator
import pickle
import warnings
warnings.filterwarnings('ignore')

# 1. Load and enhance the perpetrator's iris image
print("Loading and enhancing perpetrator's iris image...")
iris_perpetrator = cv2.imread("perpetrator_iris.bmp", cv2.IMREAD_GRAYSCALE)
enhanced_iris_perp = cv2.equalizeHist(iris_perpetrator)  # Histogram equalization

# Apply mask
mask = cv2.imread('perpetrator_mask.bmp', cv2.IMREAD_GRAYSCALE)
if mask is not None:
    enhanced_iris_perp = cv2.bitwise_and(enhanced_iris_perp, enhanced_iris_perp, mask=mask)

# Visualize enhanced iris image
plt.figure(figsize=(8, 6))
plt.imshow(enhanced_iris_perp, cmap='gray')
plt.title('Enhanced Perpetrator Iris with Mask Applied')
plt.axis('off')
plt.show()

print(f"(Perpetrator image shape): {enhanced_iris_perp.shape}")


# 2. Load preprocessed iris database
print("Loading preprocessed iris database...")
try:
    with open("iris_mask_db.pkl", "rb") as f:
        enhanced_iris, mask_iris, iris_images, iris_labels = pickle.load(f)
    print(f"Successfully loaded iris database with {len(enhanced_iris)} samples")
except:
    print("Unable to load preprocessed iris database, please ensure preprocessing step has been run")
    raise

# 3. Resize images to ensure compatibility with model's expected input
target_height, target_width = 202, 506  # Ideal dimensions calculated based on network structure

# Resize perpetrator's iris
resized_iris_perp = cv2.resize(enhanced_iris_perp, (target_width, target_height))

# Display resized image
plt.figure(figsize=(8, 6))
plt.imshow(resized_iris_perp, cmap='gray')
plt.title(f'Resized Perpetrator Iris Image ({target_height}x{target_width})')
plt.axis('off')
plt.show()

# Normalize and convert to tensor
perp_normalized = resized_iris_perp / 255.0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
perp_tensor = torch.from_numpy(perp_normalized).float().unsqueeze(0).unsqueeze(0).to(device)

# 4. Load trained encoder model
print("Loading trained encoder model...")
embed_dim = 64  # Dimension used during training

# Redefine model architecture, consistent with training
class IrisEncoder(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=4, stride=2, padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=4, stride=2, padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU()
        )
        self.linear = nn.Sequential(
            nn.Linear(32*12*31, embed_dim)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.linear(x.view(-1, 32*12*31))
        return x

model = IrisEncoder(embed_dim).to(device)

# Try to load model weights
try:
    model.load_state_dict(torch.load("best.ckpt"))
    print("Successfully loaded model weights!")
except:
    print("Warning: Unable to load model weights, using untrained model")

# 5. Get embedding vector for perpetrator's iris
with torch.no_grad():
    perp_embedding = model(perp_tensor)
    perp_embedding = perp_embedding.cpu().numpy()
    print(f"Perpetrator embedding shape: {perp_embedding.shape}")

# 6. Get embedding vectors for all suspect irises
print("Generating embedding vectors for all suspects...")
suspect_embeddings = []
suspect_ids = []

for i, img in enumerate(enhanced_iris):
    # Resize, normalize and convert to tensor
    resized = cv2.resize(img, (target_width, target_height))
    normalized = resized / 255.0
    tensor = torch.from_numpy(normalized).float().unsqueeze(0).unsqueeze(0).to(device)

    # Get embedding vector
    with torch.no_grad():
        embedding = model(tensor)
        embedding = embedding.cpu().numpy()

    suspect_embeddings.append(embedding)
    suspect_ids.append(iris_labels[i])

print(f"Processed {len(suspect_embeddings)} suspect embedding vectors")

# 7. Calculate similarity between perpetrator and each suspect
similarities = []

for i, suspect_embedding in enumerate(suspect_embeddings):
    # Calculate Euclidean distance
    perp_vec = perp_embedding[0] if perp_embedding.ndim > 1 else perp_embedding
    suspect_vec = suspect_embedding[0] if suspect_embedding.ndim > 1 else suspect_embedding

    distance = np.sqrt(np.sum((perp_vec - suspect_vec)**2))
    # Convert to similarity (closer means more similar)
    similarity = 1 / (1 + distance)

    similarities.append({
        'label': suspect_ids[i],
        'distance': distance,
        'similarity': similarity
    })


# 8. Sort by similarity (descending)
similarities = sorted(similarities, key=lambda x: x['similarity'], reverse=True)


# 9. Create similarity table (similar to fingerprint analysis table)
sim_data = {
    'label': [s['label'] for s in similarities],
    'similarity': [s['similarity'] for s in similarities],
    'distance': [s['distance'] for s in similarities]
}
sim_tb2 = pd.DataFrame(sim_data)
print("\n Similarity Table (Top 10):")
print(sim_tb2.head(10))

# 10. Improved threshold setting section - considering multiple methods
sim_values = sim_tb2['similarity'].values
dist_values = sim_tb2['distance'].values

# Plot similarity distribution
plt.figure(figsize=(10, 6))
plt.plot(range(len(sim_values)), sorted(sim_values, reverse=True), 'b-')
plt.xlabel('Index')
plt.ylabel('Similarity Score')
plt.title('Sorted Similarity Scores for Iris Comparison')
plt.grid(True)

# Try to use KneeLocator to find the "knee point" as threshold
x = range(len(sim_values))
y = sorted(sim_values, reverse=True)
kneedle = KneeLocator(x, y, S=1.0, curve='convex', direction='decreasing')
knee_point = kneedle.knee

# Consider multiple threshold setting methods, not just relying on knee point
statistical_threshold = np.mean(sim_values) + 0.5 * np.std(sim_values)
percentile_threshold = np.percentile(sim_values, 90)  # Use top 10% as threshold
fixed_threshold = 0.1745  # Experience-based fixed threshold

# Plot all possible threshold lines
knee_threshold = None
if knee_point is not None:
    knee_threshold = y[knee_point]
    plt.axvline(x=knee_point, color='r', linestyle='--',
                label=f'Knee threshold: {knee_threshold:.4f}')
plt.axhline(y=statistical_threshold, color='g', linestyle='-.',
            label=f'Statistical threshold: {statistical_threshold:.4f}')
plt.axhline(y=percentile_threshold, color='m', linestyle=':',
            label=f'Percentile threshold: {percentile_threshold:.4f}')
plt.axhline(y=fixed_threshold, color='c', linestyle='-',
            label=f'Fixed threshold: {fixed_threshold:.4f}')

plt.legend()
plt.tight_layout()
plt.show()

# Compare the effects of different threshold methods
print("\n Comparison of different threshold methods:")
threshold_methods = {
    'Knee Point': knee_threshold if knee_point is not None else None,
    'Statistical': statistical_threshold,
    'Percentile': percentile_threshold,
    'Fixed': fixed_threshold
}

for name, thresh in threshold_methods.items():
    if thresh is not None:
        matches = sim_tb2[sim_tb2['similarity'] >= thresh]
        print(f"{name} threshold ({thresh:.4f}): {len(matches)} matches")

# Select final threshold - here we consider multiple methods
# Use percentile_threshold as default as it usually provides more stable results
threshold = fixed_threshold

# 11. Determine matches based on threshold
matches = sim_tb2[sim_tb2['similarity'] >= threshold]
print(f"\nFound {len(matches)} matches using similarity threshold {threshold:.4f}:")
print(matches)

# 12. Visualize iris image sequence for best matches
# use previously defined function to display matching irises
def get_iris_image_by_label(ind):
    for i in range(len(iris_labels)):
        if iris_labels[i] == ind:
            return enhanced_iris[i]
    raise ValueError("No image with this label is found.")

# Define a function to visualize image sequences
def plot_image_sequence(images, highlight_index=None):
    if len(images) == 0:
        print("Warning: No images to display")
        return

    fig, axes = plt.subplots(1, len(images), figsize=(20, 4))

    # If there's only one image, axes won't be an array
    if len(images) == 1:
        axes = [axes]

    for i, (ax, img) in enumerate(zip(axes, images)):
        ax.imshow(img, cmap='gray')
        ax.axis('off')
        if highlight_index is not None and i == highlight_index:
            ax.set_title(f'Match #{i+1}', color='red', fontsize=14)
        else:
            ax.set_title(f'Match #{i+1}')
    plt.tight_layout()
    plt.show()

try:
    # Get iris images for matches ranked 45 to 50
    imgs = [get_iris_image_by_label(i) for i in sim_tb2['label'].values[45:50]]

    # Calculate index to highlight based on new threshold
    highlight_count = len(matches) if len(matches) < 5 else 5
    highlight_idx = highlight_count - 3  # Highlight the last matching item

    # Display image sequence
    plot_image_sequence(imgs, highlight_idx)


    print("\nHow is this possible? Iris scans don't provide a clear suspect either?")
    print("\nThere is no doubt about it now, there is an insider messing with the data!")
    print("Lucky for us, but unlucky for them, they slipped up.")
    print("We can now construct a multimodal biometric system in order to catch the perpetrator!")
except Exception as e:
    print(f"Error displaying images: {e}")

How is this possible? Iris scans don't provide a clear suspect either?

In [ ]:
def get_iris_image_by_label(ind):
    for i in range(len(iris_labels)):
        if iris_labels[i] == ind:
            return enhanced_iris[i]
    raise ValueError("No image with this label is found.")

imgs = [get_iris_image_by_label(i) for i in sim_tb2.values[:10,0]]

plot_image_sequence(imgs,kneedle.knee if kneedle.knee<=7 else 7)

There is no doubt about it now, there is an insider messing with the data! Lucky for us, but unlucky for them, they slipped up. We can now construct a multimodal biometric system in order to catch the perpetrator!

## IV Multimodal System

### 1. Score Fusion

<div class="alert alert-block alert-success">
<b>Q12: </b> Fuse your iris and fingerprint biometric system on the score level to make a prediction and solve the murder case! Use the metrics you implemented in the previous assignment to assess your system. Do you feel confident in your prediction? How do you fuse the scores? Why?
</div>

In [ ]:
# Check the structure of both tables
print("Fingerprint table columns:", sim_tb1.columns.tolist())
print("Iris table columns:", sim_tb2.columns.tolist())

# Print ID types to confirm differences
print(f"Fingerprint ID type: {type(sim_tb1['id'].iloc[0])}")
print(f"Iris ID type: {type(sim_tb2['label'].iloc[0])}")

# Convert string IDs in fingerprint table to integers to match iris table
fp_dict = {}
for i, row in sim_tb1.iterrows():
    id_val = int(row['id'])  # Convert "001" to 1
    score = row['score']
    fp_dict[id_val] = score

# Create dictionary from iris table
iris_dict = {}
for i, row in sim_tb2.iterrows():
    id_val = row['label']
    similarity = row['similarity']
    if id_val in iris_dict:
        # If ID already exists, take the maximum similarity
        iris_dict[id_val] = max(iris_dict[id_val], similarity)
    else:
        iris_dict[id_val] = similarity

# Find IDs common to both systems
common_ids = set(fp_dict.keys()).intersection(set(iris_dict.keys()))
print(f"Number of common IDs: {len(common_ids)}")

# Perform score fusion
fusion_results = []

# Extract all scores for normalization
fp_scores_list = list(fp_dict.values())
iris_scores_list = list(iris_dict.values())

# Score normalization function
def min_max_normalize(scores):
    min_val = min(scores)
    max_val = max(scores)
    range_val = max_val - min_val if max_val > min_val else 1
    return [(score - min_val) / range_val for score in scores]

# Normalize all scores
all_fp_scores_norm = min_max_normalize(fp_scores_list)
all_iris_scores_norm = min_max_normalize(iris_scores_list)

# Create dictionaries with normalized scores
fp_norm_dict = {id_val: all_fp_scores_norm[i] for i, id_val in enumerate(fp_dict.keys())}
iris_norm_dict = {id_val: all_iris_scores_norm[i] for i, id_val in enumerate(iris_dict.keys())}

# Fusion weights
w_fp = 0.5  # Fingerprint weight
w_iris = 0.5  # Iris weight

# Fuse scores for common IDs
for id_val in common_ids:
    fp_score_norm = fp_norm_dict[id_val]
    iris_score_norm = iris_norm_dict[id_val]

    # Weighted fusion
    fused_score = w_fp * fp_score_norm + w_iris * iris_score_norm

    # Save both original and normalized scores
    fusion_results.append({
        'ID': id_val,
        'Fingerprint_Score': fp_dict[id_val],
        'Iris_Score': iris_dict[id_val],
        'Fingerprint_Score_Norm': fp_score_norm,
        'Iris_Score_Norm': iris_score_norm,
        'Fused_Score': fused_score
    })

# Create DataFrame
import pandas as pd
fusion_df = pd.DataFrame(fusion_results)

# Sort by fused score in descending order
fusion_df = fusion_df.sort_values(by='Fused_Score', ascending=False).reset_index(drop=True)

# Identify most likely suspect
suspect = fusion_df.iloc[0]
print(f"\nScore Fusion Analysis Results:")
print(f"Most likely suspect ID: {suspect['ID']}")
print(f"Fusion score: {suspect['Fused_Score']:.4f}")
print(f"Original fingerprint score: {suspect['Fingerprint_Score']:.4f}")
print(f"Original iris score: {suspect['Iris_Score']:.4f}")
print(f"Normalized fingerprint score: {suspect['Fingerprint_Score_Norm']:.4f}")
print(f"Normalized iris score: {suspect['Iris_Score_Norm']:.4f}")

# Calculate score gap between first and second best
if len(fusion_df) > 1:
    second_best = fusion_df.iloc[1]
    score_gap = suspect['Fused_Score'] - second_best['Fused_Score']
    relative_gap = score_gap / suspect['Fused_Score'] * 100
    print(f"\nConfidence Analysis:")
    print(f"Score gap between 1st and 2nd: {score_gap:.4f} ({relative_gap:.2f}%)")

    if relative_gap > 20:
        confidence = "High"
    elif relative_gap > 10:
        confidence = "Medium"
    else:
        confidence = "Low"

    print(f"Confidence based on score gap: {confidence}")

# Display top 5 results
print("\nTop 5 suspects:")
print(fusion_df.head().to_string(index=True))

# Export complete results
fusion_df.to_csv('fusion_results.csv', index=False)
print("\nComplete results saved to fusion_results.csv")

# Create histogram of fusion scores
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(fusion_df['Fused_Score'], bins=20, alpha=0.7, color='skyblue')
plt.axvline(x=suspect['Fused_Score'], color='r', linestyle='--',
           label=f"Suspect (ID: {suspect['ID']})")
plt.title('Fusion Score Distribution')
plt.xlabel('Fusion Score')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.savefig('fusion_score_distribution.png')
plt.show()

# Create scatter plot showing relationship between biometric features
plt.figure(figsize=(10, 8))
plt.scatter(fusion_df['Fingerprint_Score_Norm'], fusion_df['Iris_Score_Norm'],
           alpha=0.7, c=fusion_df['Fused_Score'], cmap='viridis')
plt.colorbar(label='Fusion Score')
plt.scatter(suspect['Fingerprint_Score_Norm'], suspect['Iris_Score_Norm'],
           color='r', s=150, marker='*',
           label=f"Suspect (ID: {suspect['ID']})")
plt.title('Fingerprint Score vs Iris Score')
plt.xlabel('Normalized Fingerprint Score')
plt.ylabel('Normalized Iris Score')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.savefig('biometric_scatter_plot.png')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import auc

def min_max_normalize(scores):
    """
    Min-max normalization
    """
    min_val = min(scores)
    max_val = max(scores)
    range_val = max_val - min_val if max_val > min_val else 1
    return [(score - min_val) / range_val for score in scores]

def compute_cmc_leave_one_out(fp_dict, iris_dict, weights=(0.5, 0.5), max_rank=20):
    """
    Calculate CMC curve using leave-one-out method

    Parameters:
    fp_dict: Fingerprint score dictionary {id: score}
    iris_dict: Iris score dictionary {id: score}
    weights: Fusion weights (fingerprint weight, iris weight)
    max_rank: Maximum rank to calculate

    Returns:
    cmc_rates: Dictionary of recognition rates at each rank
    """
    # Find IDs common to both modalities
    common_ids = list(set(fp_dict.keys()).intersection(set(iris_dict.keys())))
    n = len(common_ids)
    print(f"{n} common IDs for evaluation")

    # Initialize hit counts
    rank_counts = np.zeros(max_rank)
    w_fp, w_iris = weights

    # Normalize all scores
    fp_scores_list = list(fp_dict.values())
    iris_scores_list = list(iris_dict.values())

    fp_scores_norm = min_max_normalize(fp_scores_list)
    iris_scores_norm = min_max_normalize(iris_scores_list)

    # Create normalized dictionaries
    fp_norm_dict = {id_val: fp_scores_norm[i] for i, id_val in enumerate(fp_dict.keys())}
    iris_norm_dict = {id_val: iris_scores_norm[i] for i, id_val in enumerate(iris_dict.keys())}

    # Each ID takes turn as probe
    for i, probe_id in enumerate(common_ids):
        if i % 10 == 0:
            print(f"Processing probe {i+1}/{n}...")

        # Build gallery (excluding probe)
        gallery_ids = [id_val for id_val in common_ids if id_val != probe_id]

        # Create fusion results list
        gallery_scores = []

        # Calculate fusion scores between probe and each gallery ID
        for gallery_id in gallery_ids:
            # Get normalized fingerprint and iris scores
            fp_score = fp_norm_dict[gallery_id]
            iris_score = iris_norm_dict[gallery_id]

            # Weighted fusion
            fused_score = w_fp * fp_score + w_iris * iris_score

            gallery_scores.append((gallery_id, fused_score))

        # Sort in descending order (higher scores first)
        gallery_scores.sort(key=lambda x: x[1], reverse=True)
        ranked_ids = [gid for gid, _ in gallery_scores]

        try:
            # Calculate probe's own fusion score
            probe_fp_score = fp_norm_dict[probe_id]
            probe_iris_score = iris_norm_dict[probe_id]
            probe_fused_score = w_fp * probe_fp_score + w_iris * probe_iris_score

            # Determine probe's position in the ranked list
            rank = 1
            for _, score in gallery_scores:
                if probe_fused_score > score:
                    break
                rank += 1

            # Update rank hit counts
            if rank <= max_rank:
                for r in range(rank-1, max_rank):
                    rank_counts[r] += 1

        except ValueError:
            print(f"Warning: ID {probe_id} not found in ranking list, skipping")
            continue

    # Calculate recognition rates at each rank
    cmc_rates = rank_counts / n

    # Create results dictionary
    cmc_dict = {k+1: cmc_rates[k] for k in range(max_rank)}
    return cmc_dict

def evaluate_weight_combinations(fp_dict, iris_dict, max_rank=20):
    """
    Evaluate performance of different weight combinations
    """
    weight_combinations = [
        (0.0, 1.0),  # Iris only
        (0.2, 0.8),
        (0.4, 0.6),
        (0.5, 0.5),  # Equal weights
        (0.6, 0.4),
        (0.8, 0.2),
        (1.0, 0.0)   # Fingerprint only
    ]

    results = {}
    best_weights = None
    best_rank1 = -1

    for w_fp, w_iris in weight_combinations:
        print(f"Evaluating weight combination: FP={w_fp:.1f}, Iris={w_iris:.1f}")
        weights = (w_fp, w_iris)

        # Calculate CMC
        cmc_dict = compute_cmc_leave_one_out(fp_dict, iris_dict, weights, max_rank)

        # Record key metrics
        rank1 = cmc_dict[1]
        rank5 = cmc_dict[5]
        rank10 = cmc_dict[10]

        print(f"Rank-1: {rank1:.4f}, Rank-5: {rank5:.4f}, Rank-10: {rank10:.4f}")

        # Save results
        results[weights] = {
            'cmc': cmc_dict,
            'rank1': rank1,
            'rank5': rank5,
            'rank10': rank10
        }

        # Update best weights
        if rank1 > best_rank1:
            best_rank1 = rank1
            best_weights = weights

    return results, best_weights

def plot_cmc_curves(results, best_weights, max_rank=20):
    """
    Plot CMC curves for different weight combinations
    """
    plt.figure(figsize=(12, 8))

    # Plot curve for each weight combination
    for weights, data in results.items():
        w_fp, w_iris = weights
        cmc_dict = data['cmc']
        ranks = list(range(1, max_rank+1))
        rates = [cmc_dict[r] for r in ranks]

        label = f"FP: {w_fp:.1f}, Iris: {w_iris:.1f}"
        if weights == best_weights:
            plt.plot(ranks, rates, 'o-', linewidth=3, label=f"{label} (Best)")
        else:
            plt.plot(ranks, rates, 'o-', linewidth=1.5, alpha=0.7, label=label)

    plt.title('CMC Curves Comparison for Different Fusion Weights', fontsize=14)
    plt.xlabel('Rank (k)', fontsize=12)
    plt.ylabel('Recognition Rate', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.xticks(range(1, max_rank+1, 2))
    plt.yticks(np.arange(0, 1.1, 0.1))
    plt.legend(loc='lower right', fontsize=10)
    plt.tight_layout()
    plt.savefig('cmc_weight_comparison.png', dpi=300)
    plt.show()

    # Plot Rank-1 recognition rate vs weight
    weights_fp = [w[0] for w in results.keys()]
    rank1_rates = [data['rank1'] for data in results.values()]

    plt.figure(figsize=(10, 6))
    plt.plot(weights_fp, rank1_rates, 'o-', linewidth=2, color='royalblue')
    plt.title('Fingerprint Weight vs Rank-1 Recognition Rate', fontsize=14)
    plt.xlabel('Fingerprint Weight (Iris Weight = 1 - FP Weight)', fontsize=12)
    plt.ylabel('Rank-1 Recognition Rate', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.xlim(-0.05, 1.05)
    plt.ylim(0, max(rank1_rates)*1.1)

    # Mark the best weight point
    best_fp_weight = best_weights[0]
    best_r1 = results[best_weights]['rank1']
    plt.scatter([best_fp_weight], [best_r1], color='red', s=100, zorder=5)
    plt.annotate(f'Best Weight: ({best_fp_weight:.1f}, {best_weights[1]:.1f})\nRank-1: {best_r1:.4f}',
                xy=(best_fp_weight, best_r1), xytext=(best_fp_weight+0.1, best_r1),
                arrowprops=dict(arrowstyle='->'), fontsize=10)

    plt.tight_layout()
    plt.savefig('weight_optimization.png', dpi=300)
    plt.show()

#Main program
if __name__ == "__main__":
    # Assume we load data from CSV files
    # fp_df = pd.read_csv('fingerprint_scores.csv')
    # iris_df = pd.read_csv('iris_scores.csv')

    # Build dictionaries from DataFrames
    # fp_dict = dict(zip(fp_df['id'], fp_df['score']))
    # iris_dict = dict(zip(iris_df['label'], iris_df['similarity']))

    # Set maximum rank for evaluation
    max_rank = 20

    # 1. Evaluate with default weights
    print("使用默认权重(0.5, 0.5)进行CMC评估... / Evaluating with default weights (0.5, 0.5)...")
    cmc_dict = compute_cmc_leave_one_out(fp_dict, iris_dict, weights=(0.5, 0.5), max_rank=max_rank)

    # Output recognition rates at key ranks
    for k in [1, 5, 10, 20]:
        print(f"Rank-{k}识别率 / recognition rate: {cmc_dict[k]:.4f}")

    # Plot CMC curve
    plt.figure(figsize=(10, 6))
    ranks = list(range(1, max_rank+1))
    rates = [cmc_dict[r] for r in ranks]

    plt.plot(ranks, rates, 'o-', linewidth=2)
    plt.title('Cumulative Match Characteristic (CMC) Curve', fontsize=14)
    plt.xlabel('Rank (k)', fontsize=12)
    plt.ylabel('Recognition Rate', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.xticks(range(1, max_rank+1, 2))
    plt.yticks(np.arange(0, 1.1, 0.1))

    # Calculate and display AUC
    cmc_auc = auc(ranks, rates)
    plt.text(max_rank*0.7, 0.1, f'AUC = {cmc_auc:.4f}',
             bbox=dict(facecolor='white', alpha=0.8), fontsize=12)

    plt.tight_layout()
    plt.savefig('cmc_curve.png', dpi=300)
    plt.show()

    # 2. Evaluate different weight combinations
    print("Evaluating different fusion weight combinations...")
    results, best_weights = evaluate_weight_combinations(fp_dict, iris_dict, max_rank)

    # Output best weights
    w_fp, w_iris = best_weights
    print(f"Best fusion weight combination: FP={w_fp:.2f}, Iris={w_iris:.2f}")
    print(f"Best Rank-1 recognition rate: {results[best_weights]['rank1']:.4f}")

    # 3. Plot comparison charts
    plot_cmc_curves(results, best_weights, max_rank)

    print("CMC evaluation completed! Result figures saved.")

## V. Assignment Instructions
Both a report and the implementation in this notebook have to be submitted to toledo. <u>The notebook functions as supplementary material only!</u> The report should be self contained. Feel free to add figures, code and mathematics in the report if you feel comfortable. Try to be concice and to the point!  
The assignment deadline is 28/04/2024, 11 PM. Make sure to zip your notebook, your report and any additional resources you use (data, open-source code, etc.) and name your .zip file as **Assignment2_[your student number].zip**.

1. Follow the instructions in the notebook and discuss results/impact in your report.
2. <b>Choose a number of tasks equivalent to <u>at least 6pts</u> from the list below (pts are not related to the grades): </b>
    1. [FING] OpenCV provides different KeyPoint detectors and descriptors (ORB, SIFT, SURF, BRIEF, ...). Briefly test, visually, which of these seem to extract relatively reliable and interesting points from the fingerprints dataset (you can skip the ones that require a lisence) Compare them to minutiae. (1pt)
    2. [FING and IRIS] Evaluate your biometric system in an authentication scenario, do you still aggregate the similarity scores? Why? (1pt)
    3. [FING] Play around with the gabor filter bank, what is the effect of the parameters? Can you setup a databank that results in better features? (1pt)
    4. [FING] Incorporate the orientations of the minutiae to your similarity scores and compare. (1pt)
    4. [FING] Can you find a better way to aggregate the gabor feature maps? Analyse and report. (1pt)
    5. [IRIS] Compare the performance of OpenCV Keypoint detectors to that of the triplet loss encoder in an identification setting (1pt).
    6. [FING] Try a neural network approach in order to get similarity scores directly from image data without feature extraction. (3pt)
    7. [FING or IRIS] Attempt to improve the segmentation, evaluate. (3pt)
    8. [FING] Attempt to improve the feature extraction using a completely different technique. Evaluate your results. (3pt)
    9.  [MULTI] Fuse your modalities on the feature level instead of the score level (3pt)
    10. [ . ] Create a biometric identification or authentication system on a modality of choice and evaluate it (6pt)
        - e.g. on smartphone usage patterns
        - Excludes; Fingerprints, irises and faces
    11. [ . ] Create a machine learning based identification/authentication system that will process the fingerprints and the irises simultaneously,and evaluate it (6pt)
        - e.g. based on deep learning
        - training is required, transfer learning is allowed
3. All results should be discussed in detail in your report.


<em>Note: Indicate clearly which tasks you end up choosing and where we can find the implementations and/or results. </em> <br>
<em>Note 2: Usually there are multiple valid solutions, you should always defend your approach and verbally compare it to the other approaches (possible advantages/disadvantages) in your report! If you do not provide proper reasoning we can not give you the grades you deserve! </em>

Good luck, have fun!